In [3]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
from __future__ import division

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Unable to parse whitelist (/home/hadoop/anaconda2/lib/python2.7/site-packages/moztelemetry/histogram-whitelists.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


/home/hadoop/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [24]:
# We grab pings starting from 2 days ago until [TimeWindow] days ago.
TimeWindow = 4

# Additional filters.
Filters = {
    'app': 'Firefox',
    
    # We sample 0.5% of pings. For testing, it is beter to use a small number
    # here (like 0.001) to speed up processing time.
    'fraction': 1,
    
    # Optionally restrict pings to a single channel.
     'channel': 'nightly',
}

# In case you need them!
VendorIDs = {
    'Intel': '0x8086',
    'NVIDIA': '0x10de',
    'AMD': '0x1002',
    'ATI': '0x1002'
}

In [25]:
###############################
# This section gathers pings. #
###############################

def run_get_pings():
    def fmt_date(d):
        return d.strftime("%Y%m%d")
    t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(TimeWindow + 0)) # go back 16 days
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(0)) # go back 2 days
    return get_pings(sc, build_id=(t1, t2), **Filters)

# Get pings for the parameters in the previous step.
raw_pings = run_get_pings()

In [26]:
######################################################################
# This section takes the raw ping list, then formats and filters it. #
######################################################################

# Map the pings into a more readable dictionary-like form. To see
# what these look like, execute "pings.take(1)".
unique_pings = get_pings_properties(raw_pings, [
  "clientId",
  "application/buildId",
  "environment/build/version",
  "environment/system/os/name",
  "environment/system/os/version",
  "environment/system/os/servicePackMajor",
  "environment/system/os/servicePackMinor",
  "environment/system/gfx/adapters",
  "environment/system/gfx/features",
  "payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID"
])
unique_pings = get_one_ping_per_client(unique_pings)

In [27]:
unique_pings.take(1)

[{'application/buildId': u'20160611030214',
  'clientId': u'd672bc1a-c894-4238-83c2-1a5f3696916f',
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'ATI Radeon HD 3200 Graphics (Microsoft Corporation WDDM 1.1) ',
    u'deviceID': u'0x9610',
    u'driver': u'atidxx32 atidxx64 atiumdag atiumdva atiumd64 atiumd6a atitmm64',
    u'driverDate': u'4-24-2009',
    u'driverVersion': u'8.56.1.15',
    u'subsysID': u'014f1025',
    u'vendorID': u'0x1002'}],
  'environment/system/gfx/features': {u'compositor': u'd3d11',
   u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
   u'd3d11': {u'blacklisted': False,
    u'status': u'available',
    u'textureSharing': True,
    u'version': 40960,
    u'warp': False}},
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/servicePackMajor': 0,
  'environment/system/os/servicePackMinor': 0,
  'environment/system/os/version': u'6.1',
  'payload/keyedHistograms/C

In [28]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('environment/system/gfx/adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter
    
    features = p.get('environment/system/gfx/features', None)
    if features == None:
        return None
            
    p['features'] = features
            
    # Convert the version to a tuple of integers.
    if 'driverVersion' in adapter:
        p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p

def filter_ping(p):
    return p and 'adapter' in p and 'features' in p
pings = unique_pings.map(rewrite_ping).filter(filter_ping)
pings = pings.cache()

#pings = pings.cache()

In [29]:
# Observe the format of a random ping. This may take some time since it has to
# execute the pipeline.
#pings.take(1)
#pings.take(1)
#new_pings = pings.map(rewrite_ping).filter(filter_ping)
pings.take(1)

[{'adapter': {u'GPUActive': True,
   u'description': u'ATI Radeon HD 3200 Graphics (Microsoft Corporation WDDM 1.1) ',
   u'deviceID': u'0x9610',
   u'driver': u'atidxx32 atidxx64 atiumdag atiumdva atiumd64 atiumd6a atitmm64',
   u'driverDate': u'4-24-2009',
   u'driverVersion': u'8.56.1.15',
   u'subsysID': u'014f1025',
   u'vendorID': u'0x1002'},
  'application/buildId': u'20160611030214',
  'clientId': u'd672bc1a-c894-4238-83c2-1a5f3696916f',
  'driverVersion': [8, 56, 1, 15],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'ATI Radeon HD 3200 Graphics (Microsoft Corporation WDDM 1.1) ',
    u'deviceID': u'0x9610',
    u'driver': u'atidxx32 atidxx64 atiumdag atiumdva atiumd64 atiumd6a atitmm64',
    u'driverDate': u'4-24-2009',
    u'driverVersion': u'8.56.1.15',
    u'subsysID': u'014f1025',
    u'vendorID': u'0x1002'}],
  'environment/system/gfx/features': {u'compositor': u'd3d11',
   u'd2d': {u'status': u'u

In [30]:
# Count the total number of sessions in the dataset.
TotalSessions = pings.count()
print('Number of sessions: {0}'.format(TotalSessions))

Number of sessions: 25475


In [31]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/intel.json'))
devices = {}
for gen in gpu_db['8086'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = gen[0]

In [32]:
# Sample filter #1 - how many people are using Intel devices
# with a driver less than 8.15.10.2622? (bug 1175366).
def sample_filter_1(p):
    if not 'vendorID' in p['adapter']:
        return False
    if not 'driver' in p['adapter']:
        return False

    if p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'] is None:
        return False
    #if devices[int(p['adapter']['deviceID'],16)] in ('gen6'):
    #    return True

    #if devices[int(p['adapter']['deviceID'],16)] in ('gen7'):

     #   return True
    return True

interesting_pings = pings.filter(sample_filter_1)
print(interesting_pings.count())
interesting_pings.take(1)

1010


[{'adapter': {u'GPUActive': True,
   u'RAM': 2048,
   u'description': u'AMD Radeon HD 6670',
   u'deviceID': u'0x6758',
   u'driver': u'aticfx64 aticfx64 aticfx64 aticfx32 aticfx32 aticfx32 atiumd64 atidxx64 atidxx64 atiumdag atidxx32 atidxx32 atiumdva atiumd6a atitmm64',
   u'driverDate': u'11-29-2013',
   u'driverVersion': u'13.250.18.0',
   u'subsysID': u'25571458',
   u'vendorID': u'0x1002'},
  'application/buildId': u'20160613030258',
  'clientId': u'efcdc60a-4e9f-4466-bfee-2189e4c5d1f6',
  'driverVersion': [13, 250, 18, 0],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'RAM': 2048,
    u'description': u'AMD Radeon HD 6670',
    u'deviceID': u'0x6758',
    u'driver': u'aticfx64 aticfx64 aticfx64 aticfx32 aticfx32 aticfx32 atiumd64 atidxx64 atidxx64 atiumdag atidxx32 atidxx32 atiumdva atiumd6a atitmm64',
    u'driverDate': u'11-29-2013',
    u'driverVersion': u'13.250.18.0',
    u'subsysID': u'25571458',
    u'vendorID': u

In [33]:
def webgl_fail(p):
    if not 'SUCCESS' in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID']:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)

extracted = fail_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],))
print fail_pings.count()

extracted.countByKey()


42


defaultdict(int,
            {u'FEATURE_FAILURE_EGL_LIB_INIT': 38,
             u'FEATURE_FAILURE_PERF_WARP': 2,
             u'FEATURE_FAILURE_WEBGL_DISABLED': 2})

In [34]:
fail_pings.take(5)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) 82945G Express Chipset Family',
   u'deviceID': u'0x2772',
   u'driver': u'ialmrnt5',
   u'driverDate': u'2-7-2006',
   u'driverVersion': u'6.14.10.4497',
   u'subsysID': u'00000000',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160611030214',
  'clientId': u'29ac0444-e4a3-48c8-abe2-16ce576ae222',
  'driverVersion': [6, 14, 10, 4497],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) 82945G Express Chipset Family',
    u'deviceID': u'0x2772',
    u'driver': u'ialmrnt5',
    u'driverDate': u'2-7-2006',
    u'driverVersion': u'6.14.10.4497',
    u'subsysID': u'00000000',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
   u'd3d11': {u'status': u'blacklisted'}},
  'environment/system/os/name': u'Windows_NT',
  'environment/system/o

In [37]:
win7_pings = fail_pings.filter(lambda p: float(p['environment/system/os/version']) > 5.1)
win7_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()


defaultdict(int,
            {u'FEATURE_FAILURE_EGL_LIB_INIT': 4,
             u'FEATURE_FAILURE_PERF_WARP': 2,
             u'FEATURE_FAILURE_WEBGL_DISABLED': 2})

In [41]:
def expected_dis(p):
    k = p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()
    if k == ['FEATURE_FAILURE_PERF_WARP']:
        return False
    if k == ['FEATURE_FAILURE_WEBGL_DISABLED']:
        return False
    return True
interesting_win7 = win7_pings.filter(expected_dis)
interesting_win7.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()


defaultdict(int, {u'FEATURE_FAILURE_EGL_LIB_INIT': 4})

In [42]:
interesting_win7.take(4)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) G33/G31 Express Chipset Family (Microsoft Corporation - WDDM 1.0)',
   u'deviceID': u'0x29c2',
   u'driver': u'igdumd32',
   u'driverDate': u'10-1-2012',
   u'driverVersion': u'8.15.10.2697',
   u'subsysID': u'29c28086',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160614030210',
  'clientId': u'3f0a6b58-eeb5-4c1d-b06d-6dd78a22f35c',
  'driverVersion': [8, 15, 10, 2697],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) G33/G31 Express Chipset Family (Microsoft Corporation - WDDM 1.0)',
    u'deviceID': u'0x29c2',
    u'driver': u'igdumd32',
    u'driverDate': u'10-1-2012',
    u'driverVersion': u'8.15.10.2697',
    u'subsysID': u'29c28086',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'failed', u'version': u'1.1'},
   u'd3d11': {u'status': u'failed'}},
  'env

In [43]:
xp_pings = fail_pings.filter(lambda p: float(p['environment/system/os/version']) == 5.1)
xp_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()

defaultdict(int, {u'FEATURE_FAILURE_EGL_LIB_INIT': 34})

In [44]:
xp_pings.map(lambda p: (p['adapter']['vendorID'],)).countByKey()

defaultdict(int,
            {u'0x0000': 5,
             u'0x1002': 8,
             u'0x10de': 2,
             u'0x1106': 1,
             u'0x15ad': 1,
             u'0x8086': 17})

In [53]:
not_vga_pings.map(lambda p: (p['adapter']['vendorID'],)).countByKey()

defaultdict(int, {u'0x1002': 5, u'0x10de': 6, u'0x8086': 16})

In [45]:
xp_intel_pings = xp_pings.filter(lambda p: p['adapter']['vendorID'] == '0x8086')
def do_gen(p):
    p['gen'] = devices[int(p['adapter']['deviceID'],16)]
    return p
    
xp_intel_pings = xp_intel_pings.map(do_gen)
xp_intel_pings.map(lambda p: (p['gen'],)).countByKey()

defaultdict(int, {u'gen2': 2, u'gen3': 10, u'gen4': 5})

In [55]:
intel_pings = not_vga_pings.filter(lambda p: p['adapter']['vendorID'] == '0x8086')
nv_pings = not_vga_pings.filter(lambda p: p['adapter']['vendorID'] == '0x10de')
def do_gen(p):
    p['gen'] = devices[int(p['adapter']['deviceID'],16)]
    return p
    
intel_pings = intel_pings.map(do_gen)
intel_pings.map(lambda p: (p['gen'],)).countByKey()

defaultdict(int, {u'gen2': 1, u'gen3': 12, u'gen4': 3})

In [42]:
nv_pings.take(4)

[{'adapter': {u'GPUActive': True,
   u'RAM': 4095,
   u'description': u'NVIDIA GeForce GTX 970',
   u'deviceID': u'0x13c2',
   u'driver': u'nvd3dumx,nvwgf2umx,nvwgf2umx,nvwgf2umx nvd3dum,nvwgf2um,nvwgf2um,nvwgf2um',
   u'driverDate': u'4-27-2016',
   u'driverVersion': u'10.18.13.6510',
   u'subsysID': u'39753842',
   u'vendorID': u'0x10de'},
  'clientId': u'5a171441-eeba-4e82-a356-50120314b974',
  'driverVersion': [10, 18, 13, 6510],
  'environment/build/version': u'49.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'RAM': 4095,
    u'description': u'NVIDIA GeForce GTX 970',
    u'deviceID': u'0x13c2',
    u'driver': u'nvd3dumx,nvwgf2umx,nvwgf2umx,nvwgf2umx nvd3dum,nvwgf2um,nvwgf2um,nvwgf2um',
    u'driverDate': u'4-27-2016',
    u'driverVersion': u'10.18.13.6510',
    u'subsysID': u'39753842',
    u'vendorID': u'0x10de'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'failureId': u'OK', u'status': u'failed', u'version': u'1.1'},
   

In [57]:
intel_pings.filter(lambda p: p['gen'] == 'gen4').take(5)

[{'adapter': {u'GPUActive': True,
   u'description': u'Mobile Intel(R) 4 Series Express Chipset Family',
   u'deviceID': u'0x2a42',
   u'driver': u'igxprd32',
   u'driverDate': u'8-25-2008',
   u'driverVersion': u'6.14.10.4980',
   u'subsysID': u'024d1028',
   u'vendorID': u'0x8086'},
  'clientId': u'4b055f9c-aed1-4f50-88c8-cff8fc7d72e1',
  'driverVersion': [6, 14, 10, 4980],
  'environment/build/version': u'49.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Mobile Intel(R) 4 Series Express Chipset Family',
    u'deviceID': u'0x2a42',
    u'driver': u'igxprd32',
    u'driverDate': u'8-25-2008',
    u'driverVersion': u'6.14.10.4980',
    u'subsysID': u'024d1028',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'failureId': u'FEATURE_FAILURE_DL_BLACKLIST_g984',
    u'status': u'unavailable',
    u'version': u'1.1'},
   u'd3d11': {u'failureId': u'FEATURE_FAILURE_BUG_1153381',
    u'status': u'

In [ ]:
def webgl_fail(p):
    if p['payload/histograms/CANVAS_WEBGL_FAILURE_ID']['SUCCESS'] == 0:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return True


     #   return True
    return False
vga_pings = fail_pings.filter(vga_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)


amd1_pings = fail_pings.filter(lambda p: p['features']['webgl']['failureId'] == 'FEATURE_FAILURE_AMD1')



#TDRResults = sample_result_1.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)
#help(pd.Series(TDRResults.values, DeviceResetReason))
#print(pd.Series(TDRResults.values[1:8],DeviceResetReason))
#pd.Series(TDRResults.values[1:8],DeviceResetReason).plot.bar()
#reduce(lambda x, y: sample_result_1.take(2))
#sample_result_1.take(1)[0]['payload/histograms/DEVICE_RESET_REASON'])
#fail_pings.take(1)
def match_count(a, b):
    print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        a.count(),
        b.count(),
       ((a.count() / b.count()) * 100)))



def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p['adapter'][sourceKey],)
    return data.map(extract).countByKey()    
match_count(interesting_pings, pings)    
match_count(fail_pings, interesting_pings)
match_count(vga_pings, fail_pings)
match_count(not_vga_pings, fail_pings)
match_count(amd1_pings, fail_pings)
#map_x_to_count(not_vga_pings, 'driverVersion')
#not_vga_pings.take(5)

#fail_pings.map(lambda p: (p['features']['webgl']['failureId'],)).countByKey()
#amd1_pings.take(23)
amd1_pings.map(lambda p: (p['adapter']['driverVersion'],)).countByKey()